## Phase 1: Sorting it out

In [64]:
import pandas as pd
from pathlib import Path

# Notebook is in: cpi_project/notebooks/
# Excel is in:    cpi_project/data/raw/2025.xlsx
BASE_DIR = Path.cwd().parent          # goes from notebooks/ → cpi_project/
FILE_PATH = BASE_DIR / "data" / "raw" / "2025.xlsx"
SHEET_NAME = "2025"
YEAR = 2025
SOURCE = "DOS"

print("Current folder:", Path.cwd())
print("Excel path:", FILE_PATH)
print("File exists?", FILE_PATH.exists())


Current folder: C:\Users\User\cpi_project\notebooks
Excel path: C:\Users\User\cpi_project\data\raw\2025.xlsx
File exists? True


In [65]:
df_raw = pd.read_excel(FILE_PATH, sheet_name=SHEET_NAME, header=None)

print("Shape (rows, cols):", df_raw.shape)
df_raw.head(10)


Shape (rows, cols): (48, 16)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,الأرقام القياسية الشهرية والتراكمية لأسعار الم...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Monthly And cummulative Consumer Price Indices...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,مجموعـــــــــات الانفـــــــاق,الاهمية النسبية,كانون ثاني,شباط,اذار,نيسان,ايار,حزيران,تموز,آب,ايلول,تشرين اول,تشرين ثاني,كانون اول,المعدل,Expenditure Groups
3,NaN,Relative Imp.,Jan.,Feb.,Mar.,Apr.,May,Jun.,Jul.,Aug.,Sep.,Oct.,Nov.,Dec.,Average,NaN
4,جميع المواد,100,112.234774,112.358883,112.425127,112.526678,112.766094,112.982383,112.868804,112.632466,112.738584,112.819669,112.46483,113.224992,112.670274,All Items
5,1) الاغذية والمشروبات غير الكحولية,26.523947,109.876346,109.751204,109.639604,110.623567,111.54121,111.218243,109.949614,109.182313,109.596942,109.025625,107.95455,108.521592,109.740068,1)Food and non-Alcoholic Beverages
6,الغذاء,23.796409,110.518394,110.340248,110.291861,111.118658,112.072358,111.68756,110.245066,109.347982,109.73248,109.026662,107.791684,108.39726,110.047518,Food Items
7,الحبوب ومنتجاتها,4.171318,118.841873,118.725954,118.694335,118.595747,118.570376,118.421671,118.372697,118.419612,118.394564,118.390116,118.341967,117.889179,118.471507,Cereals and Products
8,اللحوم والدواجن,4.686294,119.3296,120.487506,119.301258,117.718649,122.371528,123.569023,118.463304,114.44944,110.794308,104.947666,102.686762,106.59922,115.059855,Meat and Poultry
9,الاسماك ومنتجات البحر,0.414004,103.702077,104.756308,105.73604,105.040966,103.470461,103.77179,103.348934,103.001812,102.306969,105.35706,105.243654,104.87245,104.217377,Fish and Sea Products


In [66]:
# Let's inspect a specific row that usually contains the month names in your file
HEADER_ROW = 3

print("Row", HEADER_ROW, "preview:")
df_raw.iloc[HEADER_ROW, :16]


Row 3 preview:


0               NaN
1     Relative Imp.
2              Jan.
3              Feb.
4              Mar.
5              Apr.
6               May
7              Jun.
8              Jul.
9              Aug.
10             Sep.
11             Oct.
12             Nov.
13             Dec.
14          Average
15              NaN
Name: 3, dtype: object

In [67]:
# Months are expected in columns 2..13 (12 months)
months = df_raw.iloc[HEADER_ROW, 2:14].tolist()
print("Months detected:", months)

DATA_START_ROW = HEADER_ROW + 1  # data starts after header row

# Take only the part we need (16 columns based on your file layout)
df = df_raw.iloc[DATA_START_ROW:, :16].copy()

df.columns = [
    "category_ar",
    "relative_weight",
    *[f"m_{m}" for m in months],  # month columns (we prefix with m_)
    "avg_annual",
    "category_en"
]

# Keep rows that actually have an English category name
df = df[df["category_en"].notna()].copy()

df.head(10)


Months detected: ['Jan.', 'Feb.', 'Mar.', 'Apr.', 'May', 'Jun.', 'Jul.', 'Aug.', 'Sep.', 'Oct.', 'Nov.', 'Dec.']


,category_ar,relative_weight,m_Jan.,m_Feb.,m_Mar.,m_Apr.,m_May,m_Jun.,m_Jul.,m_Aug.,m_Sep.,m_Oct.,m_Nov.,m_Dec.,avg_annual,category_en
4,جميع المواد,100,112.234774,112.358883,112.425127,112.526678,112.766094,112.982383,112.868804,112.632466,112.738584,112.819669,112.46483,113.224992,112.670274,All Items
5,1) الاغذية والمشروبات غير الكحولية,26.523947,109.876346,109.751204,109.639604,110.623567,111.54121,111.218243,109.949614,109.182313,109.596942,109.025625,107.95455,108.521592,109.740068,1)Food and non-Alcoholic Beverages
6,الغذاء,23.796409,110.518394,110.340248,110.291861,111.118658,112.072358,111.68756,110.245066,109.347982,109.73248,109.026662,107.791684,108.39726,110.047518,Food Items
7,الحبوب ومنتجاتها,4.171318,118.841873,118.725954,118.694335,118.595747,118.570376,118.421671,118.372697,118.419612,118.394564,118.390116,118.341967,117.889179,118.471507,Cereals and Products
8,اللحوم والدواجن,4.686294,119.3296,120.487506,119.301258,117.718649,122.371528,123.569023,118.463304,114.44944,110.794308,104.947666,102.686762,106.59922,115.059855,Meat and Poultry
9,الاسماك ومنتجات البحر,0.414004,103.702077,104.756308,105.73604,105.040966,103.470461,103.77179,103.348934,103.001812,102.306969,105.35706,105.243654,104.87245,104.217377,Fish and Sea Products
10,الالبان ومنتجاتها والبيض,3.720077,111.542874,111.626305,111.011286,110.55037,111.340553,111.628622,111.523047,111.527163,111.657077,111.742651,111.866233,111.69901,111.476266,Dairy Products and Eggs
11,الزيوت والدهون,1.696951,117.488551,117.335655,117.182644,117.68961,117.50715,117.686818,117.575137,117.738594,118.031461,125.029058,129.296629,132.226167,120.398956,Oils and Fats
12,الفواكه والمكسرات,2.566837,99.925749,100.263432,103.713596,113.494161,116.74191,114.079469,111.538546,106.054627,106.520721,105.677719,104.91453,104.746515,107.305915,Fruits and Nuts
13,الخضروات والبقول الجافة والمعلبة,2.962259,90.501231,87.412908,87.763357,88.702477,86.632246,84.837705,83.385186,85.970891,92.538392,92.446684,85.714485,83.282924,87.432374,Vegetables and Legumes Dry and Canned


## Phase 2: Clean categories & build parent/child hierarchy

In [68]:
import re
import numpy as np


In [69]:
def is_main_group(name):
    """
    Returns True if the category name starts with a number like '1)'
    """
    if pd.isna(name):
        return False
    return bool(re.match(r"^\s*\d+\)", str(name)))


In [70]:
def clean_category_name(name):
    """
    Removes leading numbering like '1)' from category names
    """
    if pd.isna(name):
        return name
    return re.sub(r"^\s*\d+\)\s*", "", str(name)).strip()


In [71]:
df["is_main"] = df["category_en"].apply(is_main_group)

df["category_name_en"] = df["category_en"].apply(clean_category_name)
df["category_name_ar"] = df["category_ar"].astype(str).str.strip()

df[["category_en", "category_name_en", "is_main"]].head(12)



,category_en,category_name_en,is_main
4,All Items,All Items,False
5,1)Food and non-Alcoholic Beverages,Food and non-Alcoholic Beverages,True
6,Food Items,Food Items,False
7,Cereals and Products,Cereals and Products,False
8,Meat and Poultry,Meat and Poultry,False
9,Fish and Sea Products,Fish and Sea Products,False
10,Dairy Products and Eggs,Dairy Products and Eggs,False
11,Oils and Fats,Oils and Fats,False
12,Fruits and Nuts,Fruits and Nuts,False
13,Vegetables and Legumes Dry and Canned,Vegetables and Legumes Dry and Canned,False


In [72]:
df["category_level"] = np.where(
    df["category_name_en"].str.lower() == "all items",
    "all_items",
    np.where(df["is_main"], "main", "sub")
)

df["category_level"].value_counts()


category_level
sub          31
main         12
all_items     1
Name: count, dtype: int64

In [73]:
current_main = None
parents = []

for _, row in df.iterrows():
    if row["category_level"] == "all_items":
        parents.append(None)
        current_main = None
    elif row["category_level"] == "main":
        parents.append("All Items")
        current_main = row["category_name_en"]
    else:
        parents.append(current_main)

df["parent_name_en"] = parents


In [74]:
invalid = df[
    (df["category_level"] == "sub") &
    (df["parent_name_en"].isna())
]

invalid[["category_name_en"]].head()


,category_name_en


In [75]:
df[[
    "category_name_en",
    "category_level",
    "parent_name_en",
    "relative_weight"
]].head(20)


,category_name_en,category_level,parent_name_en,relative_weight
4,All Items,all_items,NaN,100
5,Food and non-Alcoholic Beverages,main,All Items,26.523947
6,Food Items,sub,Food and non-Alcoholic Beverages,23.796409
7,Cereals and Products,sub,Food and non-Alcoholic Beverages,4.171318
8,Meat and Poultry,sub,Food and non-Alcoholic Beverages,4.686294
9,Fish and Sea Products,sub,Food and non-Alcoholic Beverages,0.414004
10,Dairy Products and Eggs,sub,Food and non-Alcoholic Beverages,3.720077
11,Oils and Fats,sub,Food and non-Alcoholic Beverages,1.696951
12,Fruits and Nuts,sub,Food and non-Alcoholic Beverages,2.566837
13,Vegetables and Legumes Dry and Canned,sub,Food and non-Alcoholic Beverages,2.962259


In [76]:
df = df.reset_index(drop=True)  # ensure df has a clean 0..n-1 index
df["category_order"] = df.index + 1
df[["category_name_en", "category_order"]].head(10)

,category_name_en,category_order
0,All Items,1
1,Food and non-Alcoholic Beverages,2
2,Food Items,3
3,Cereals and Products,4
4,Meat and Poultry,5
5,Fish and Sea Products,6
6,Dairy Products and Eggs,7
7,Oils and Fats,8
8,Fruits and Nuts,9
9,Vegetables and Legumes Dry and Canned,10


# Phase 3: Reshaping the Data

In [77]:
month_cols = [col for col in df.columns if col.startswith("m_")]
month_cols


['m_Jan.',
 'm_Feb.',
 'm_Mar.',
 'm_Apr.',
 'm_May',
 'm_Jun.',
 'm_Jul.',
 'm_Aug.',
 'm_Sep.',
 'm_Oct.',
 'm_Nov.',
 'm_Dec.']

In [78]:
df_long = df.melt(
    id_vars=[
        "category_name_en",
        "category_name_ar",
        "category_level",
        "parent_name_en",
        "relative_weight"
    ],
    value_vars=month_cols,
    var_name="month_label",
    value_name="cpi_index"
)


In [79]:
df_long.head(12) #for inspection


,category_name_en,category_name_ar,category_level,parent_name_en,relative_weight,month_label,cpi_index
0,All Items,جميع المواد,all_items,NaN,100,m_Jan.,112.234774
1,Food and non-Alcoholic Beverages,1) الاغذية والمشروبات غير الكحولية,main,All Items,26.523947,m_Jan.,109.876346
2,Food Items,الغذاء,sub,Food and non-Alcoholic Beverages,23.796409,m_Jan.,110.518394
3,Cereals and Products,الحبوب ومنتجاتها,sub,Food and non-Alcoholic Beverages,4.171318,m_Jan.,118.841873
4,Meat and Poultry,اللحوم والدواجن,sub,Food and non-Alcoholic Beverages,4.686294,m_Jan.,119.3296
5,Fish and Sea Products,الاسماك ومنتجات البحر,sub,Food and non-Alcoholic Beverages,0.414004,m_Jan.,103.702077
6,Dairy Products and Eggs,الالبان ومنتجاتها والبيض,sub,Food and non-Alcoholic Beverages,3.720077,m_Jan.,111.542874
7,Oils and Fats,الزيوت والدهون,sub,Food and non-Alcoholic Beverages,1.696951,m_Jan.,117.488551
8,Fruits and Nuts,الفواكه والمكسرات,sub,Food and non-Alcoholic Beverages,2.566837,m_Jan.,99.925749
9,Vegetables and Legumes Dry and Canned,الخضروات والبقول الجافة والمعلبة,sub,Food and non-Alcoholic Beverages,2.962259,m_Jan.,90.501231


In [80]:
df_long["cpi_index"] = pd.to_numeric(df_long["cpi_index"], errors="coerce")
df_long["relative_weight"] = pd.to_numeric(df_long["relative_weight"], errors="coerce")


In [81]:
df_long["month_text"] = df_long["month_label"].str.replace("m_", "", regex=False)
df_long[["month_label", "month_text"]].head()

,month_label,month_text
0,m_Jan.,Jan.
1,m_Jan.,Jan.
2,m_Jan.,Jan.
3,m_Jan.,Jan.
4,m_Jan.,Jan.


In [82]:
month_map = {
    "Jan.": 1, "Feb.": 2, "Mar.": 3, "Apr.": 4,
    "May": 5, "Jun.": 6, "Jul.": 7, "Aug.": 8,
    "Sep.": 9, "Oct.": 10, "Nov.": 11, "Dec.": 12
}

df_long["month"] = df_long["month_text"].map(month_map)


In [83]:
df_long["year"] = YEAR
df_long["date_id"] = df_long["year"] * 100 + df_long["month"]


In [84]:
df_long.duplicated(
    subset=["date_id", "category_name_en"]
).sum()

np.int64(0)

In [85]:
df_long["cpi_index"].isna().mean()


np.float64(0.0)

In [86]:
df_long[["date_id", "category_name_en", "cpi_index", "relative_weight"]].head(10)

,date_id,category_name_en,cpi_index,relative_weight
0,202501,All Items,112.234774,100.000000
1,202501,Food and non-Alcoholic Beverages,109.876346,26.523947
2,202501,Food Items,110.518394,23.796409
3,202501,Cereals and Products,118.841873,4.171318
4,202501,Meat and Poultry,119.329600,4.686294
5,202501,Fish and Sea Products,103.702077,0.414004
6,202501,Dairy Products and Eggs,111.542874,3.720077
7,202501,Oils and Fats,117.488551,1.696951
8,202501,Fruits and Nuts,99.925749,2.566837
9,202501,Vegetables and Legumes Dry and Canned,90.501231,2.962259


# Phase 4: Build dim tables + fact table

In [87]:
from pathlib import Path

OUTPUT_DIR = BASE_DIR / "output"
OUTPUT_DIR.mkdir(exist_ok=True)

print("Output folder:", OUTPUT_DIR)


Output folder: C:\Users\User\cpi_project\output


In [88]:
import pandas as pd

dim_date = (
    df_long[["date_id", "year", "month"]]
    .drop_duplicates()
    .sort_values(["year", "month"])
    .copy()
)

# Month name for readability (optional but nice)
month_name_map = {
    1:"Jan", 2:"Feb", 3:"Mar", 4:"Apr", 5:"May", 6:"Jun",
    7:"Jul", 8:"Aug", 9:"Sep", 10:"Oct", 11:"Nov", 12:"Dec"
}
dim_date["month_name"] = dim_date["month"].map(month_name_map)

# month_start and month_end (helps Power BI time intelligence later)
dim_date["month_start"] = pd.to_datetime(dim_date["year"].astype(str) + "-" + dim_date["month"].astype(str) + "-01")
dim_date["month_end"] = (dim_date["month_start"] + pd.offsets.MonthEnd(1))

# store as date (not datetime)
dim_date["month_start"] = dim_date["month_start"].dt.date
dim_date["month_end"] = dim_date["month_end"].dt.date

dim_date.head()


,date_id,year,month,month_name,month_start,month_end
0,202501,2025,1,Jan,2025-01-01,2025-01-31
44,202502,2025,2,Feb,2025-02-01,2025-02-28
88,202503,2025,3,Mar,2025-03-01,2025-03-31
132,202504,2025,4,Apr,2025-04-01,2025-04-30
176,202505,2025,5,May,2025-05-01,2025-05-31


In [89]:
# 1) Take unique categories from df (not df_long) because df has one row per category, and create a simple surrogate key
dim_category = (
    df[["category_name_en", "category_name_ar", "category_level", "parent_name_en", "category_order"]]
    .drop_duplicates(subset=["category_name_en"]).copy()
    .sort_values("category_order")
    .reset_index(drop=True)
)



dim_category["category_id"] = dim_category.index + 1  # starts at 1
dim_category["category_order"] = dim_category["category_order"]

# 2) Map parent_name_en -> parent_category_id using a lookup
name_to_id = dict(zip(dim_category["category_name_en"], dim_category["category_id"]))
dim_category["parent_category_id"] = dim_category["parent_name_en"].map(name_to_id)

# 3) Essential flags (default false for now)
dim_category["is_essential_core"] = False
dim_category["is_essential_extended"] = False

# 4) Governance fields
dim_category["source"] = SOURCE
dim_category["is_active"] = True

dim_category.head(10)


,category_name_en,category_name_ar,category_level,parent_name_en,category_order,category_id,parent_category_id,is_essential_core,is_essential_extended,source,is_active
0,All Items,جميع المواد,all_items,NaN,1,1,NaN,False,False,DOS,True
1,Food and non-Alcoholic Beverages,1) الاغذية والمشروبات غير الكحولية,main,All Items,2,2,1.0,False,False,DOS,True
2,Food Items,الغذاء,sub,Food and non-Alcoholic Beverages,3,3,2.0,False,False,DOS,True
3,Cereals and Products,الحبوب ومنتجاتها,sub,Food and non-Alcoholic Beverages,4,4,2.0,False,False,DOS,True
4,Meat and Poultry,اللحوم والدواجن,sub,Food and non-Alcoholic Beverages,5,5,2.0,False,False,DOS,True
5,Fish and Sea Products,الاسماك ومنتجات البحر,sub,Food and non-Alcoholic Beverages,6,6,2.0,False,False,DOS,True
6,Dairy Products and Eggs,الالبان ومنتجاتها والبيض,sub,Food and non-Alcoholic Beverages,7,7,2.0,False,False,DOS,True
7,Oils and Fats,الزيوت والدهون,sub,Food and non-Alcoholic Beverages,8,8,2.0,False,False,DOS,True
8,Fruits and Nuts,الفواكه والمكسرات,sub,Food and non-Alcoholic Beverages,9,9,2.0,False,False,DOS,True
9,Vegetables and Legumes Dry and Canned,الخضروات والبقول الجافة والمعلبة,sub,Food and non-Alcoholic Beverages,10,10,2.0,False,False,DOS,True


In [90]:
# sub categories must have a parent
orphans = dim_category[
    (dim_category["category_level"] == "sub") &
    (dim_category["parent_category_id"].isna())
]

print("Orphan subcategories:", len(orphans))
orphans[["category_name_en", "parent_name_en"]].head()


Orphan subcategories: 0


,category_name_en,parent_name_en


In [91]:
fact_cpi = df_long[["date_id", "category_name_en", "cpi_index", "relative_weight"]].copy()

# map to category_id
fact_cpi["category_id"] = fact_cpi["category_name_en"].map(name_to_id)

# basic governance
fact_cpi["source"] = SOURCE

# keep only the columns we want in the warehouse fact table
fact_cpi = fact_cpi[["date_id", "category_id", "cpi_index", "relative_weight", "source"]]

fact_cpi.head(10)


,date_id,category_id,cpi_index,relative_weight,source
0,202501,1,112.234774,100.000000,DOS
1,202501,2,109.876346,26.523947,DOS
2,202501,3,110.518394,23.796409,DOS
3,202501,4,118.841873,4.171318,DOS
4,202501,5,119.329600,4.686294,DOS
5,202501,6,103.702077,0.414004,DOS
6,202501,7,111.542874,3.720077,DOS
7,202501,8,117.488551,1.696951,DOS
8,202501,9,99.925749,2.566837,DOS
9,202501,10,90.501231,2.962259,DOS


In [92]:
missing_cat = fact_cpi["category_id"].isna().sum()
dupes = fact_cpi.duplicated(subset=["date_id", "category_id"]).sum()
expected = len(dim_category) * len(dim_date)

print("Missing category_id:", missing_cat)
print("Duplicate (date_id, category_id):", dupes)
print("Fact rows:", len(fact_cpi), "Expected:", expected)


Missing category_id: 0
Duplicate (date_id, category_id): 0
Fact rows: 528 Expected: 528


In [93]:
dim_date.to_csv(OUTPUT_DIR / "dim_date.csv", index=False)
dim_category.to_csv(OUTPUT_DIR / "dim_category.csv", index=False)
fact_cpi.to_csv(OUTPUT_DIR / "fact_cpi.csv", index=False)

# Phase 5 : Exporting

In [94]:
from pathlib import Path

OUTPUT_DIR = BASE_DIR / "output"
OUTPUT_DIR.mkdir(exist_ok=True)

dim_date_path = OUTPUT_DIR / "dim_date.csv"
dim_category_path = OUTPUT_DIR / "dim_category.csv"
fact_cpi_path = OUTPUT_DIR / "fact_cpi.csv"

# If you still have parent_name_en, keep it in Python but DON'T export it to Postgres
dim_category_export = dim_category.drop(columns=["parent_name_en"], errors="ignore")

dim_date.to_csv(dim_date_path, index=False)
dim_category_export.to_csv(dim_category_path, index=False)
fact_cpi.to_csv(fact_cpi_path, index=False)

print("Saved:", dim_date_path)
print("Saved:", dim_category_path)
print("Saved:", fact_cpi_path)


Saved: C:\Users\User\cpi_project\output\dim_date.csv
Saved: C:\Users\User\cpi_project\output\dim_category.csv
Saved: C:\Users\User\cpi_project\output\fact_cpi.csv


In [96]:
dim_category_export.columns.tolist()


['category_name_en',
 'category_name_ar',
 'category_level',
 'category_order',
 'category_id',
 'parent_category_id',
 'is_essential_core',
 'is_essential_extended',
 'source',
 'is_active']

In [98]:
import pandas as pd

dim_category_pg = dim_category.copy()

# Force integer-with-null support (pandas nullable integer)
dim_category_pg["category_id"] = dim_category_pg["category_id"].astype("Int64")
dim_category_pg["parent_category_id"] = dim_category_pg["parent_category_id"].astype("Int64")
dim_category_pg["category_order"] = dim_category_pg["category_order"].astype("Int64")

# Reorder columns to match SQL import
dim_category_pg = dim_category_pg[
    [
        "category_name_en",
        "category_name_ar",
        "category_level",
        "category_order",
        "category_id",
        "parent_category_id",
        "is_essential_core",
        "is_essential_extended",
        "source",
        "is_active",
    ]
].copy()

# Export new clean file
dim_category_pg_path = OUTPUT_DIR / "dim_category_pg.csv"
dim_category_pg.to_csv(dim_category_pg_path, index=False)

print("Saved:", dim_category_pg_path.resolve())
dim_category_pg.head(5)


Saved: C:\Users\User\cpi_project\output\dim_category_pg.csv


,category_name_en,category_name_ar,category_level,category_order,category_id,parent_category_id,is_essential_core,is_essential_extended,source,is_active
0,All Items,جميع المواد,all_items,1,1,<NA>,False,False,DOS,True
1,Food and non-Alcoholic Beverages,1) الاغذية والمشروبات غير الكحولية,main,2,2,1,False,False,DOS,True
2,Food Items,الغذاء,sub,3,3,2,False,False,DOS,True
3,Cereals and Products,الحبوب ومنتجاتها,sub,4,4,2,False,False,DOS,True
4,Meat and Poultry,اللحوم والدواجن,sub,5,5,2,False,False,DOS,True
